In [1]:
%reload_ext autoreload
%autoreload 2

import h5py
N_Cs = 9

with h5py.File('../dxtb/dxtb-gpu/gpu-cpu_analysis/rdkit/alkanes_data_500.hdf5', 'r') as f:
    for mol_name, data in f.items():
        if mol_name == f"alkane_{N_Cs}_carbons":
            atomic_numbers = data['atomic_numbers'][:]
            coordinates = data['coordinates'][:]

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")

Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29


In [2]:
import dxtb
from dxtb._src.typing import DD
import torch
from dxtb.config import ConfigCache

batch_size = 64

print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")
print(f"batch_size: {batch_size}")

dd = {"device": torch.device("cuda:0"), "dtype": torch.float32}
numbers = torch.tensor(atomic_numbers, device=dd["device"], dtype=torch.int32)
positions = torch.tensor(coordinates, device=dd["device"], dtype=dd["dtype"])
numbers = torch.stack([numbers] * batch_size)
positions = torch.stack([positions] * batch_size).requires_grad_()
charges = torch.zeros((batch_size,), device=dd["device"], dtype=dd["dtype"])

results = {}

Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29
batch_size: 64


In [3]:
opts = {"scf_mode": "full", "batch_mode": 2, "int_driver": "libcint", "maxiter":10000}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)
dxtb.timer.reset()
e = calc.get_energy(positions, chrg=charges)
dxtb.timer.start("Forces autograd")
forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
dxtb.timer.stop("Forces autograd")
dxtb.timer.print(v=0)

results[f"e_{opts['scf_mode']}"] = e
results[f"forces_{opts['scf_mode']}"] = forces
results[f"Fgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.cache["fock"].sum(), positions, retain_graph=True)[0]
results[f"Pgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.get_density(positions, chrg=charges).sum(), positions, retain_graph=True)[0]

# For reconnect modes
scf_charges = calc.get_charges(positions, chrg=charges)
scf_charge_mode = opts["scf_mode"]



Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.044           2.66
 - Halogen                  0.005          12.00
 - Repulsion                0.026          59.87
 - DispersionD3             0.012          27.85
Integrals                   0.197          12.01
 - Overlap                  0.194          98.35
 - Core Hamiltonian         0.003           1.65
SCF                         1.185          72.18
 - Interaction Cache        0.001           0.08
 - Potential                0.011           0.90
 - Fock build               0.001           0.09
 - Diagonalize              1.085          91.59
 - Density                  0.003           0.26
 - Charges                  0.003           0.27
Forces autograd             0.134           8.16
------------------------------------------------
Sum                         1.560          95.01
Total                       1.642         100.00


In [4]:
opts = {"scf_mode": "implicit", "batch_mode": 2, "int_driver": "libcint", "maxiter":10000}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)
dxtb.timer.reset()
e = calc.get_energy(positions, chrg=charges)
dxtb.timer.start("Forces autograd")
forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
dxtb.timer.stop("Forces autograd")
dxtb.timer.print(v=0)

results[f"e_{opts['scf_mode']}"] = e
results[f"forces_{opts['scf_mode']}"] = forces
results[f"Fgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.cache["fock"].sum(), positions, retain_graph=True)[0]
results[f"Pgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.get_density(positions, chrg=charges).sum(), positions, retain_graph=True)[0]

# For reconnect modes
scf_charges = calc.get_charges(positions, chrg=charges)
scf_charge_mode = opts["scf_mode"]




Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.009           1.67
 - Repulsion                0.001           6.81
 - Halogen                  0.004          39.99
 - DispersionD3             0.005          52.79
Integrals                   0.186          34.26
 - Overlap                  0.184          99.06
 - Core Hamiltonian         0.002           0.94
SCF                         0.202          37.22
 - Interaction Cache        0.001           0.44
 - Potential                0.154          76.14
 - Fock build               0.001           0.65
 - Diagonalize              0.115          56.96
 - Density                  0.006           3.03
 - Charges                  0.006           3.07
Forces autograd             0.145          26.73
------------------------------------------------
Sum                         0.542          99.87
Total                       0.543         100.00


In [5]:
opts = {"scf_mode": "reconnectxt", "batch_mode": 1, "int_driver": "libcint", "maxiter": 1}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)
dxtb.timer.reset()
e = calc.get_energy(positions, chrg=charges, scf_charges_guess=scf_charges)
dxtb.timer.start("Forces autograd")
forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
dxtb.timer.stop("Forces autograd")
dxtb.timer.print(v=0)

results[f"e_{opts['scf_mode']}"] = e
results[f"forces_{opts['scf_mode']}"] = forces
results[f"Fgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.cache["fock"].sum(), positions, retain_graph=True)[0]
results[f"Pgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.get_density(positions, chrg=charges, scf_charges_guess=scf_charges).sum(), positions, retain_graph=True)[0]




Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.007           2.66
 - Repulsion                0.001           9.03
 - Halogen                  0.003          52.74
 - DispersionD3             0.002          37.68
Integrals                   0.156          63.07
 - Overlap                  0.154          98.95
 - Core Hamiltonian         0.002           1.05
SCF                         0.014           5.67
 - Interaction Cache        0.001           5.14
 - Potential                0.010          69.53
 - Fock build               0.000           0.73
 - Diagonalize              0.007          50.11
 - Density                  0.000           3.00
 - Charges                  0.000           3.11
Forces autograd             0.070          28.32
------------------------------------------------
Sum                         0.246          99.72
Total                       0.247         100.00


In [6]:
opts = {"scf_mode": "reconnect", "batch_mode": 1, "int_driver": "libcint", "maxiter": 1}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, **dd, opts=opts, timer=True)
calc.opts.cache = ConfigCache(enabled=False, density=True, fock=True, overlap=False)
dxtb.timer.reset()
e = calc.get_energy(positions, chrg=charges, scf_charges_guess=scf_charges)
dxtb.timer.start("Forces autograd")
forces = torch.autograd.grad(sum(e), positions, retain_graph=True)[0]
dxtb.timer.stop("Forces autograd")
dxtb.timer.print(v=0)

results[f"e_{opts['scf_mode']}"] = e
results[f"forces_{opts['scf_mode']}"] = forces
results[f"Fgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.cache["fock"].sum(), positions, retain_graph=True)[0]
results[f"Pgrad_{opts['scf_mode']}"] = torch.autograd.grad(calc.get_density(positions, chrg=charges, scf_charges_guess=scf_charges).sum(), positions, retain_graph=True)[0]



Timings
-------

Objective                Time (s)        % Total
------------------------------------------------
Classicals                  0.007           2.16
 - Halogen                  0.003          50.90
 - Repulsion                0.001           8.64
 - DispersionD3             0.003          39.93
Integrals                   0.156          49.04
 - Overlap                  0.154          99.00
 - Core Hamiltonian         0.002           1.00
SCF                         0.006           2.02
 - Interaction Cache        0.001          13.17
 - Potential                0.003          52.80
 - Fock build               0.000           0.53
 - Diagonalize              0.002          37.33
 - Density                  0.000           2.45
 - Charges                  0.000           2.53
Forces autograd             0.148          46.56
------------------------------------------------
Sum                         0.317          99.78
Total                       0.318         100.00


In [7]:
print(f"Number of carbon atoms in {mol_name}: {N_Cs}")
print(f"Nb of atoms: {len(atomic_numbers)}")
print(f"batch_size: {batch_size}")
print(f"Using precomputed {scf_charge_mode} charges")

print()
for scf_mode in ["full", "implicit", "reconnect", "reconnectxt"]:
    print(f"{results[f'e_{scf_mode}'][0]} = e_{scf_mode}")

print()
for scf_mode in ["full", "implicit", "reconnect", "reconnectxt"]:
    print(f"{(results[f'e_{scf_mode}'][0] - results['e_implicit'][0]):.2e} = e_{scf_mode} - e_implicit")

print()
for scf_mode in ["full", "reconnect", "reconnectxt"]:
    print(f"{(results[f'forces_{scf_mode}'][0] - results['forces_full'][0]).max():.2e} = max(forces_{scf_mode} - forces_full)")
    print(f"{(results[f'forces_{scf_mode}'][0] - results['forces_implicit'][0]).max():.2e} = max(forces_{scf_mode} - forces_implicit)")

print()
for scf_mode in ["full", "reconnect", "reconnectxt"]:
    print(f"{(results[f'Fgrad_{scf_mode}'][0] - results['Fgrad_full'][0]).max():.2e} = max(Fgrad_{scf_mode} - Fgrad_full)")
    print(f"{(results[f'Fgrad_{scf_mode}'][0] - results['Fgrad_implicit'][0]).max():.2e} = max(Fgrad_{scf_mode} - Fgrad_implicit)")

print()
for scf_mode in ["full", "reconnect", "reconnectxt"]:
    print(f"{(results[f'Pgrad_{scf_mode}'][0] - results['Pgrad_full'][0]).max():.2e} = max(Pgrad_{scf_mode} - Pgrad_full)")
    print(f"{(results[f'Pgrad_{scf_mode}'][0] - results['Pgrad_implicit'][0]).max():.2e} = max(Pgrad_{scf_mode} - Pgrad_implicit)")


Number of carbon atoms in alkane_9_carbons: 9
Nb of atoms: 29
batch_size: 64
Using precomputed implicit charges

-3.555391550064087 = e_full
-3.55527925491333 = e_implicit
-3.555271625518799 = e_reconnect
-3.5552761554718018 = e_reconnectxt

-1.12e-04 = e_full - e_implicit
0.00e+00 = e_implicit - e_implicit
7.63e-06 = e_reconnect - e_implicit
3.10e-06 = e_reconnectxt - e_implicit

0.00e+00 = max(forces_full - forces_full)
4.77e-06 = max(forces_full - forces_implicit)
6.56e-06 = max(forces_reconnect - forces_full)
8.82e-06 = max(forces_reconnect - forces_implicit)
6.20e-06 = max(forces_reconnectxt - forces_full)
3.34e-06 = max(forces_reconnectxt - forces_implicit)

0.00e+00 = max(Fgrad_full - Fgrad_full)
4.74e+00 = max(Fgrad_full - Fgrad_implicit)
5.25e+00 = max(Fgrad_reconnect - Fgrad_full)
9.32e+00 = max(Fgrad_reconnect - Fgrad_implicit)
4.04e+00 = max(Fgrad_reconnectxt - Fgrad_full)
2.38e-05 = max(Fgrad_reconnectxt - Fgrad_implicit)

0.00e+00 = max(Pgrad_full - Pgrad_full)
4.72e-01 =